In [ ]:
!pip install transformers
!pip install SentencePiece

     |████████████████████████████████| 2.5MB 8.1MB/s 
     |████████████████████████████████| 901kB 38.4MB/s 
     |████████████████████████████████| 3.3MB 51.1MB/s 
     |████████████████████████████████| 1.2MB 9.2MB/s 


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import nltk
import numpy as np
import pandas as pd

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

In [ ]:
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu

def bleu(ref, gen):
    ''' 
    calculate pair wise bleu score. uses nltk implementation
    Args:
        references : a list of reference sentences 
        candidates : a list of candidate(generated) sentences
    Returns:
        bleu score(float)
    '''
    ref_bleu = []
    gen_bleu = []
    for l in gen:
        gen_bleu.append(l.split())
    for i,l in enumerate(ref):
        ref_bleu.append([l.split()])
    cc = SmoothingFunction()
    score_bleu = corpus_bleu(ref_bleu, gen_bleu, weights=(0, 1, 0, 0), smoothing_function=cc.method4)
    return score_bleu

In [ ]:
df = pd.read_excel('Data_SOC.xlsx')
df = pd.DataFrame(df)
df1 = pd.DataFrame(columns={'Finance','Sc_Literature','Music/Entertainment'})
df1.head()
len(df)

10

In [ ]:
i = 0
while( i != len(df)):
  content = df['Sc_Literature'][i]
  input_ids = tokenizer.encode(content[:100], return_tensors='pt')
  output = model.generate(input_ids, max_length = 1000, do_sample = True, top_k = 50, top_p = 0.95)

  reference = tokenizer.decode(output[0], skip_special_tokens= True).split()
  
  len1 = len(content)
  len2 = len(reference)
  if(len1 >= len2):
    hypothesis = content.split()[:len2]
  else:
    reference = reference[:len1]
    hypothesis = content.split()

  BLEUscore = nltk.translate.bleu_score.corpus_bleu([reference], [hypothesis])
  df1.loc[i,'Sc_Literature'] = BLEUscore
  print(BLEUscore)
  i += 1

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.47587330964125224
0.3742031646082125
0
0
0
0.3521856535823236
0
0
0.408248290463863
0.47897362544357464


In [ ]:
i = 0
while( i != len(df)):
  content = df['Finance'][i]
  input_ids = tokenizer.encode(content[:100], return_tensors='pt')
  output = model.generate(input_ids, max_length = 1000, do_sample = True, top_k = 50, top_p = 0.95)

  reference = tokenizer.decode(output[0], skip_special_tokens= True).split()
  
  len1 = len(content)
  len2 = len(reference)
  if(len1 >= len2):
    hypothesis = content.split()[:len2]
  else:
    reference = reference[:len1]
    hypothesis = content.split()

  BLEUscore = nltk.translate.bleu_score.corpus_bleu([reference], [hypothesis])
  df1.loc[i,'Finance'] = BLEUscore
  print(BLEUscore)
  i += 1

0


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.3495274586585512
0
0
0.3655552228545123
0.40276720463657734
0
0
0.3882726777522233
0.4347208719449914


In [ ]:
i = 0
while( i != len(df)):
  content = df['Music/Entertainment'][i]
  input_ids = tokenizer.encode(content[:100], return_tensors='pt')
  output = model.generate(input_ids, max_length = 1000, do_sample = True, top_k = 50, top_p = 0.95)

  reference = tokenizer.decode(output[0], skip_special_tokens= True).split()

  len1 = len(content)
  len2 = len(reference)
  if(len1 >= len2):
    hypothesis = content.split()[:len2]
  else:
    reference = reference[:len1]
    hypothesis = content.split()

  BLEUscore = nltk.translate.bleu_score.corpus_bleu([reference], [hypothesis])
  df1.loc[i,'Music/Entertainment'] = BLEUscore
  print(BLEUscore)
  i += 1

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.40546144983876986
0.46173663094410267
0.47587330964125224
0.3742031646082125
0
0.46439804737625334
0.34095107969299543
0
0
0.4172261448611506


In [ ]:
df1.to_csv(r'BLEU_Scores_T5.csv', index=False)